# Data Processing: From Raw Files to Final Dataset

This notebook processes raw stock data and creates the final dataset for machine learning.

**Steps:**
1. Load and clean raw stock files (AAPL, TSLA, MSFT)
2. Calculate technical indicators
3. Add sentiment features
4. Combine everything into final_dataset.csv

In [9]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("STOCK DATA PROCESSING PIPELINE")
print("="*80)


STOCK DATA PROCESSING PIPELINE


In [10]:
def calculate_rsi(data, window=14):
    """Calculate Relative Strength Index"""
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def process_stock(filepath, ticker_name):
    """Process a single stock file with all technical indicators"""
    print(f"\n{'='*60}")
    print(f"Processing {ticker_name}")
    print('='*60)
    
    # Load raw data
    df = pd.read_csv(filepath)
    print(f"   Loaded: {len(df)} rows")
    
    # Clean data - remove $ symbols
    df['Close'] = df['Close/Last'].str.replace('$', '').astype(float)
    df['Open'] = df['Open'].str.replace('$', '').astype(float)
    df['High'] = df['High'].str.replace('$', '').astype(float)
    df['Low'] = df['Low'].str.replace('$', '').astype(float)
    
    # Drop the old column
    df = df.drop('Close/Last', axis=1)
    
    # Convert date and sort
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date').reset_index(drop=True)
    
    print(f"   Date range: {df['Date'].min().date()} to {df['Date'].max().date()}")
    
    # Calculate technical indicators
    print("   Calculating technical indicators...")
    
    # RSI
    df['RSI'] = calculate_rsi(df['Close'])
    
    # EMA
    df['EMA_20'] = df['Close'].ewm(span=20, adjust=False).mean()
    df['EMA_50'] = df['Close'].ewm(span=50, adjust=False).mean()
    
    # MACD
    ema_fast = df['Close'].ewm(span=12, adjust=False).mean()
    ema_slow = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = ema_fast - ema_slow
    df['MACD_Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    df['MACD_Hist'] = df['MACD'] - df['MACD_Signal']
    
    # Bollinger Bands
    sma = df['Close'].rolling(window=20).mean()
    std = df['Close'].rolling(window=20).std()
    df['BB_Upper'] = sma + (std * 2)
    df['BB_Middle'] = sma
    df['BB_Lower'] = sma - (std * 2)
    df['BB_Width'] = df['BB_Upper'] - df['BB_Lower']
    
    # Price metrics
    df['Price_Change'] = df['Close'].pct_change()
    df['Price_Direction'] = (df['Close'].shift(-1) > df['Close']).astype(int)
    df['Volatility_20'] = df['Close'].pct_change().rolling(window=20).std()
    df['Daily_Return'] = (df['Close'] - df['Open']) / df['Open']
    df['High_Low_Range'] = (df['High'] - df['Low']) / df['Low']
    
    # Add ticker column
    df['Ticker'] = ticker_name
    
    # Remove NaN rows (from rolling calculations)
    original_len = len(df)
    df = df.dropna()
    
    print(f"   Removed {original_len - len(df)} NaN rows")
    print(f"   Final: {len(df)} rows with {len(df.columns)} columns")
    
    return df

print("✅ Functions defined")

✅ Functions defined


In [12]:
# Process each stock
print("\n" + "="*80)
print("STEP 1: PROCESSING INDIVIDUAL STOCKS")
print("="*80)

# UPDATE THESE PATHS TO YOUR ACTUAL FILE LOCATIONS
aapl = process_stock('/Users/aryan/Desktop/Stock-Price-Volatility-Sentiment-ML/data/AAPL.csv', 'AAPL')
tsla = process_stock('/Users/aryan/Desktop/Stock-Price-Volatility-Sentiment-ML/data/TSLA.csv', 'TSLA')
msft = process_stock('/Users/aryan/Desktop/Stock-Price-Volatility-Sentiment-ML/data/MSFT.csv', 'MSFT')

print("\n✅ All stocks processed successfully!")


# Save combined stocks (before adding sentiment)
print("\n" + "="*80)
print("SAVING COMBINED STOCKS FILE")
print("="*80)
# Save combined stocks (before adding sentiment)
print("\n" + "="*80)
print("SAVING COMBINED STOCKS FILE")
print("="*80)

# Save in the main data folder (no 'processed' subfolder needed)
combined_stocks_path = '/Users/aryan/Desktop/Stock-Price-Volatility-Sentiment-ML/data/all_stocks_combined.csv'
all_stocks.to_csv(combined_stocks_path, index=False)

print(f"✅ Saved: all_stocks_combined.csv")
print(f"   Location: {combined_stocks_path}")
print(f"   Rows: {len(all_stocks):,}")
print(f"   Columns: {len(all_stocks.columns)}")



STEP 1: PROCESSING INDIVIDUAL STOCKS

Processing AAPL
   Loaded: 1255 rows
   Date range: 2020-12-02 to 2025-12-01
   Calculating technical indicators...
   Removed 20 NaN rows
   Final: 1235 rows with 22 columns

Processing TSLA
   Loaded: 1255 rows
   Date range: 2020-12-02 to 2025-12-01
   Calculating technical indicators...
   Removed 20 NaN rows
   Final: 1235 rows with 22 columns

Processing MSFT
   Loaded: 1255 rows
   Date range: 2020-12-02 to 2025-12-01
   Calculating technical indicators...
   Removed 20 NaN rows
   Final: 1235 rows with 22 columns

✅ All stocks processed successfully!

SAVING COMBINED STOCKS FILE

SAVING COMBINED STOCKS FILE
✅ Saved: all_stocks_combined.csv
   Location: /Users/aryan/Desktop/Stock-Price-Volatility-Sentiment-ML/data/all_stocks_combined.csv
   Rows: 3,705
   Columns: 26


In [13]:
# Combine all stocks
print("\n" + "="*80)
print("STEP 2: COMBINING ALL STOCKS")
print("="*80)

all_stocks = pd.concat([aapl, tsla, msft], ignore_index=True)

print(f"\nCombined dataset:")
print(f"   Total rows: {len(all_stocks):,}")
print(f"   AAPL: {len(aapl):,} rows")
print(f"   TSLA: {len(tsla):,} rows")
print(f"   MSFT: {len(msft):,} rows")

print("\nSample of combined data:")
print(all_stocks[['Date', 'Ticker', 'Close', 'RSI', 'Price_Direction']].head(10))

print("\n✅ Stocks combined successfully!")



STEP 2: COMBINING ALL STOCKS

Combined dataset:
   Total rows: 3,705
   AAPL: 1,235 rows
   TSLA: 1,235 rows
   MSFT: 1,235 rows

Sample of combined data:
        Date Ticker   Close        RSI  Price_Direction
0 2020-12-31   AAPL  132.69  67.870651                0
1 2021-01-04   AAPL  129.41  62.114919                1
2 2021-01-05   AAPL  131.01  65.455459                0
3 2021-01-06   AAPL  126.60  47.728080                1
4 2021-01-07   AAPL  130.92  54.796422                1
5 2021-01-08   AAPL  132.05  55.128598                0
6 2021-01-11   AAPL  128.98  53.451091                0
7 2021-01-12   AAPL  128.80  50.882626                1
8 2021-01-13   AAPL  130.89  48.389196                0
9 2021-01-14   AAPL  128.91  46.775716                0

✅ Stocks combined successfully!


In [14]:
# Add sentiment features
print("\n" + "="*80)
print("STEP 3: ADDING SENTIMENT FEATURES")
print("="*80)

# Load sentiment data
# UPDATE THIS PATH TO YOUR SENTIMENT FILE
sentiment_df = pd.read_csv('/Users/aryan/Desktop/Stock-Price-Volatility-Sentiment-ML/data/Sentiment_Stock_data.csv')

print(f"Sentiment data loaded: {len(sentiment_df):,} sentences")

# Calculate sentiment statistics
total_sentences = len(sentiment_df)
positive_sentences = (sentiment_df['Sentiment'] == 1).sum()
negative_sentences = (sentiment_df['Sentiment'] == 0).sum()

sentiment_score = positive_sentences / total_sentences

print(f"\nSentiment Analysis:")
print(f"   Positive: {positive_sentences:,} ({positive_sentences/total_sentences*100:.1f}%)")
print(f"   Negative: {negative_sentences:,} ({negative_sentences/total_sentences*100:.1f}%)")
print(f"   Overall score: {sentiment_score:.3f}")

# Add sentiment features to stock data
all_stocks['Sentiment_Score'] = sentiment_score
all_stocks['Sentiment_Positive_Ratio'] = positive_sentences / total_sentences
all_stocks['Sentiment_Negative_Ratio'] = negative_sentences / total_sentences

# Add simulated daily sentiment variation
np.random.seed(42)
all_stocks['Daily_Sentiment'] = np.random.normal(sentiment_score, 0.1, len(all_stocks))
all_stocks['Daily_Sentiment'] = all_stocks['Daily_Sentiment'].clip(0, 1)

print(f"\n✅ Added 4 sentiment features")


STEP 3: ADDING SENTIMENT FEATURES
Sentiment data loaded: 108,751 sentences

Sentiment Analysis:
   Positive: 55,725 (51.2%)
   Negative: 53,026 (48.8%)
   Overall score: 0.512

✅ Added 4 sentiment features


In [19]:
# Add sentiment features from ALL sentiment files
print("\n" + "="*80)
print("STEP 3: ADDING SENTIMENT FEATURES FROM ALL FILES")
print("="*80)

# === FILE 1: Sentiment_Stock_data.csv ===
print("\n[File 1] Loading Sentiment_Stock_data.csv...")
sentiment_df1 = pd.read_csv('/Users/aryan/Desktop/Stock-Price-Volatility-Sentiment-ML/data/Sentiment_Stock_data.csv')
print(f"   Loaded: {len(sentiment_df1):,} sentences")

total_1 = len(sentiment_df1)
positive_1 = (sentiment_df1['Sentiment'] == 1).sum()
negative_1 = (sentiment_df1['Sentiment'] == 0).sum()
sentiment_score_1 = positive_1 / total_1

print(f"   Positive: {positive_1:,} ({positive_1/total_1*100:.1f}%)")
print(f"   Negative: {negative_1:,} ({negative_1/total_1*100:.1f}%)")
print(f"   Score: {sentiment_score_1:.3f}")


# === FILE 2: all-data__1_.csv ===
print("\n[File 2] Loading all-data__1_.csv...")
sentiment_df2 = pd.read_csv('/Users/aryan/Desktop/Stock-Price-Volatility-Sentiment-ML/data/all-data.csv', encoding='latin1')
print(f"   Loaded: {len(sentiment_df2):,} sentences")

# This file has 'neutral', 'positive', 'negative' as labels
# The first column contains the sentiment label
sentiment_col = sentiment_df2.columns[0]  # Get first column name
sentiment_counts = sentiment_df2[sentiment_col].value_counts()

print(f"   Distribution:")
for label, count in sentiment_counts.items():
    print(f"      {label}: {count:,} ({count/len(sentiment_df2)*100:.1f}%)")

# Calculate sentiment score (positive ratio)
positive_2 = sentiment_counts.get('positive', 0)
negative_2 = sentiment_counts.get('negative', 0)
neutral_2 = sentiment_counts.get('neutral', 0)
total_2 = len(sentiment_df2)
sentiment_score_2 = positive_2 / total_2 if total_2 > 0 else 0.5

print(f"   Score: {sentiment_score_2:.3f}")


# === FILE 3: Data.csv ===
print("\n[File 3] Loading Data.csv...")
sentiment_df3 = pd.read_csv('/Users/aryan/Desktop/Stock-Price-Volatility-Sentiment-ML/data/Data.csv', encoding='latin1')
print(f"   Loaded: {len(sentiment_df3):,} rows")

# This file has news headlines with 'Label' column (0/1)
total_3 = len(sentiment_df3)
positive_3 = (sentiment_df3['Label'] == 1).sum()
negative_3 = (sentiment_df3['Label'] == 0).sum()
sentiment_score_3 = positive_3 / total_3

print(f"   Positive (1): {positive_3:,} ({positive_3/total_3*100:.1f}%)")
print(f"   Negative (0): {negative_3:,} ({negative_3/total_3*100:.1f}%)")
print(f"   Score: {sentiment_score_3:.3f}")
print(f"   Date range: {sentiment_df3['Date'].min()} to {sentiment_df3['Date'].max()}")


# === COMBINE ALL SENTIMENT SCORES ===
print("\n" + "="*60)
print("COMBINED SENTIMENT ANALYSIS")
print("="*60)

# Calculate weighted average sentiment
total_sentences = total_1 + total_2 + total_3
weighted_sentiment = (
    (sentiment_score_1 * total_1 + 
     sentiment_score_2 * total_2 + 
     sentiment_score_3 * total_3) / total_sentences
)

print(f"\nTotal sentiment data points: {total_sentences:,}")
print(f"   From Sentiment_Stock_data.csv: {total_1:,}")
print(f"   From all-data__1_.csv: {total_2:,}")
print(f"   From Data.csv: {total_3:,}")

print(f"\nIndividual sentiment scores:")
print(f"   File 1 score: {sentiment_score_1:.3f}")
print(f"   File 2 score: {sentiment_score_2:.3f}")
print(f"   File 3 score: {sentiment_score_3:.3f}")

print(f"\nWeighted average sentiment: {weighted_sentiment:.3f}")


# === ADD FEATURES TO STOCK DATA ===
print("\n" + "="*60)
print("ADDING SENTIMENT FEATURES TO STOCK DATA")
print("="*60)

# Add overall sentiment features
all_stocks['Sentiment_Score'] = weighted_sentiment
all_stocks['Sentiment_File1_Score'] = sentiment_score_1
all_stocks['Sentiment_File2_Score'] = sentiment_score_2
all_stocks['Sentiment_File3_Score'] = sentiment_score_3

# Calculate positive/negative ratios from all files
total_positive = positive_1 + positive_2 + positive_3
total_negative = negative_1 + negative_2 + negative_3
total_neutral = neutral_2  # Only File 2 has neutral

all_stocks['Sentiment_Positive_Ratio'] = total_positive / total_sentences
all_stocks['Sentiment_Negative_Ratio'] = total_negative / total_sentences
all_stocks['Sentiment_Neutral_Ratio'] = total_neutral / total_sentences

# Add simulated daily sentiment variation
np.random.seed(42)
all_stocks['Daily_Sentiment'] = np.random.normal(weighted_sentiment, 0.1, len(all_stocks))
all_stocks['Daily_Sentiment'] = all_stocks['Daily_Sentiment'].clip(0, 1)

print(f"\n✅ Added 8 sentiment features to all {len(all_stocks):,} rows")
print("\nSentiment features added:")
print("   1. Sentiment_Score (weighted average)")
print("   2. Sentiment_File1_Score")
print("   3. Sentiment_File2_Score")
print("   4. Sentiment_File3_Score")
print("   5. Sentiment_Positive_Ratio")
print("   6. Sentiment_Negative_Ratio")
print("   7. Sentiment_Neutral_Ratio")
print("   8. Daily_Sentiment (with variation)")


STEP 3: ADDING SENTIMENT FEATURES FROM ALL FILES

[File 1] Loading Sentiment_Stock_data.csv...
   Loaded: 108,751 sentences
   Positive: 55,725 (51.2%)
   Negative: 53,026 (48.8%)
   Score: 0.512

[File 2] Loading all-data__1_.csv...
   Loaded: 4,845 sentences
   Distribution:
      neutral: 2,878 (59.4%)
      positive: 1,363 (28.1%)
      negative: 604 (12.5%)
   Score: 0.281

[File 3] Loading Data.csv...
   Loaded: 4,101 rows
   Positive (1): 2,166 (52.8%)
   Negative (0): 1,935 (47.2%)
   Score: 0.528
   Date range: 2000-01-03 to 2016-07-01

COMBINED SENTIMENT ANALYSIS

Total sentiment data points: 117,697
   From Sentiment_Stock_data.csv: 108,751
   From all-data__1_.csv: 4,845
   From Data.csv: 4,101

Individual sentiment scores:
   File 1 score: 0.512
   File 2 score: 0.281
   File 3 score: 0.528

Weighted average sentiment: 0.503

ADDING SENTIMENT FEATURES TO STOCK DATA

✅ Added 8 sentiment features to all 3,705 rows

Sentiment features added:
   1. Sentiment_Score (weighted a

In [20]:
# Create final dataset
print("\n" + "="*80)
print("STEP 4: CREATING FINAL DATASET")
print("="*80)

# Select and order columns
feature_columns = [
    'Date', 'Ticker',
    # Price features
    'Close', 'Open', 'High', 'Low', 'Volume',
    # Technical indicators
    'RSI', 'EMA_20', 'EMA_50', 
    'MACD', 'MACD_Signal', 'MACD_Hist',
    'BB_Upper', 'BB_Middle', 'BB_Lower', 'BB_Width',
    # Price metrics
    'Price_Change', 'Volatility_20', 'Daily_Return', 'High_Low_Range',
    # Sentiment features
    'Sentiment_Score', 'Sentiment_Positive_Ratio', 
    'Sentiment_Negative_Ratio', 'Daily_Sentiment',
    # Target
    'Price_Direction'
]

final_df = all_stocks[feature_columns].copy()

# Final check for NaN
print(f"Before final NaN check: {len(final_df)} rows")
final_df = final_df.dropna()
print(f"After final NaN check: {len(final_df)} rows")

# Save final dataset
# UPDATE THIS PATH
output_path = '/Users/aryan/Desktop/Stock-Price-Volatility-Sentiment-ML/data/final_dataset.csv'
final_df.to_csv(output_path, index=False)

print(f"\n✅ FINAL DATASET SAVED!")
print(f"   Location: {output_path}")
print(f"   Rows: {len(final_df):,}")
print(f"   Columns: {len(final_df.columns)}")


STEP 4: CREATING FINAL DATASET
Before final NaN check: 3705 rows
After final NaN check: 3705 rows

✅ FINAL DATASET SAVED!
   Location: /Users/aryan/Desktop/Stock-Price-Volatility-Sentiment-ML/data/final_dataset.csv
   Rows: 3,705
   Columns: 26
